Laboratorio 4 - Modelo de Regresión Logística Polinomial

By Cristian Laynez - 201281

El objetivo de este laboratorio es crear un modelo de regresión logística polinomial que prediga fielmente si un paciente sufrirá de un paro cardíaco.


Información Extra: 
para la data set proveida donde se muestran las condiciones físicas y contextuales para más de 4000 pacientes de enfermedades cardíacas. El dataset relaciona a cada paciente con una etiqueta (1 = tuvo un paro cardíaco, 0 = no tuvo paro cardíaco)

In [ ]:
Task 1.1 - Leer archivo proporcionado y almacenarlo en un np.array

Task 1.2 - Ajustar modelo logístico polinomial (Tomando en cuenta la variable dependietne de salida [sufre o no subre un paro cardíaco])

Task 1.3 - Utilizar implementación vectorial del algoritmo de regresioń logística

Task 1.4 - Determinar el grado del polinomio que mejor describe la nube de puntos (cross-validation)

Task 1.5 - Análisis sobre sus hallazgos